In [25]:
import pandas as pd

with open("./data/protein_abundance.tsv", "r") as h:
    prot_abundance = {}
    for line in h.readlines():
        if line.startswith("ID"): continue
        prot_abundance[line.split()[0]] = float(line.split()[1])

prot_abundance


{'3055.EDO98358': 18111.0,
 '3055.EDP00188': 12748.0,
 '3055.EDP02473': 12589.0,
 '3055.EDP02283': 12246.0,
 '3055.DAA00950': 10900.0,
 '3055.EDO96901': 10432.0,
 '3055.EDP03051': 10367.0,
 '3055.EDP04241': 8524.0,
 '3055.EDP03062': 8208.0,
 '3055.EDO95759': 8103.0,
 '3055.EDP01264': 7911.0,
 '3055.EDP03517': 7794.0,
 '3055.EDP01611': 7589.0,
 '3055.EDP02901': 7405.0,
 '3055.DAA00941': 6958.0,
 '3055.EDP08545': 6847.0,
 '3055.EDP00448': 6691.0,
 '3055.DAA01471': 6369.0,
 '3055.EDO96058': 6359.0,
 '3055.EDO99452': 6242.0,
 '3055.EDO96805': 6129.0,
 '3055.EDP04043': 6114.0,
 '3055.DAA00966': 5887.0,
 '3055.DAA00933': 5631.0,
 '3055.EDO98566': 5314.0,
 '3055.EDO99874': 5173.0,
 '3055.EDO96390': 5008.0,
 '3055.EDO97626': 4962.0,
 '3055.DAA00958': 4953.0,
 '3055.DAA00957': 4544.0,
 '3055.EDP01593': 4516.0,
 '3055.DAA00964': 4485.0,
 '3055.EDP01838': 4352.0,
 '3055.EDO98586': 4270.0,
 '3055.EDO98239': 4258.0,
 '3055.DAA00967': 4171.0,
 '3055.DAA00904': 4105.0,
 '3055.EDP01087': 4086.0,
 '305

In [22]:
import requests

def convert_to_chlamydomonas_id(paxdb_id):
    url = f"https://rest.uniprot.org/uniprotkb/search?fields=xref_gramene&format=tsv&query=%28{paxdb_id}%29&size=500"
    response = requests.get(url)
    
    if response.status_code == 200:
        gram = response.text.split("\n")[1].replace(";", "")
        return gram
    
    else:
        print(f"Error retrieving UniProt entry for PaxDB ID {paxdb_id}")
        return None
    

convert_to_chlamydomonas_id("3055.EDO98358")

'PNW89085'

In [ ]:

mRNA_list = []
prot_list = []

for i in prot_abundance.keys():
    paxdb_id = i
    chlamy_id = convert_to_chlamydomonas_id(paxdb_id)
    prot = prot_abundance[i]
    
    print(chlamy_id, prot)
    
    if chlamy_id: 
        mRNA_list.append(chlamy_id)
        prot_list.append(prot)


In [49]:
with open("C:\CRAGjobs\YangLabIntern\codon_optimization\pytorch-transformer\data\idmapping_2023_07_20.tsv", "r") as h:
    mydic = {}
    for line in h.readlines():
        if line.startswith("From"): continue
        if len(line.split()) < 3: continue
        mydic[line.split()[0]] = line.split()[2].replace(";", "")

mapping = pd.DataFrame.from_dict(mydic, orient="index")
mapping.columns =  ["Gramene"]
mapping


,Gramene
3055.EDO98358,PNW89085
3055.EDP00188,PNW84742
3055.EDP03051,PNW76070
3055.EDP04241,PNW84147
3055.EDP03062,PNW76019
...,...
3055.EDO98310,PNW89074
3055.EDO96340,PNW70381
3055.EDP05833,PNW77460
3055.EDP04993,PNW83014


In [50]:

abundance_values = pd.DataFrame.from_dict(prot_abundance, orient="index")
abundance_values.columns = ["Abundance"]
abundance_values

,Abundance
3055.EDO98358,18111.00
3055.EDP00188,12748.00
3055.EDP02473,12589.00
3055.EDP02283,12246.00
3055.DAA00950,10900.00
...,...
3055.EDP04993,0.02
3055.EDP08552,0.02
3055.EDP05984,0.02
3055.EDO98570,0.02


In [52]:
combined = mapping.merge(abundance_values, left_index=True, right_index=True)
combined

,Gramene,Abundance
3055.EDO98358,PNW89085,18111.00
3055.EDP00188,PNW84742,12748.00
3055.EDP03051,PNW76070,10367.00
3055.EDP04241,PNW84147,8524.00
3055.EDP03062,PNW76019,8208.00
...,...,...
3055.EDO98310,PNW89074,0.12
3055.EDO96340,PNW70381,0.08
3055.EDP05833,PNW77460,0.03
3055.EDP04993,PNW83014,0.02


In [59]:
with open("C:\CRAGjobs\YangLabIntern\codon_optimization\pytorch-transformer\data\chlamydomonas_mRNA.fa", "r") as h:
    chlamy_mRNA = {}
    for line in h.readlines():
        if line.startswith(">"):
            key = line.split()[0].replace(">", "")
            chlamy_mRNA[key] = ""
            # print(key)
        else:
            chlamy_mRNA[key] += line.strip()
            # print(line.strip())

chlamy = pd.DataFrame.from_dict(chlamy_mRNA, orient="index")
chlamy.columns = ["mRNA"]
chlamy

,mRNA
PNW88964,ATGCCGATCAACAGGCAGAAAATCCTTCAGCTGGCCAGCAGCTTCC...
PNW89007,ATGAACGTGAAACGCCGCAAAGGCCCGTTGGTTGAGGCCTTGAGAG...
PNW89008,ATGAACGTGAAACGCCGCAAAGGCCCGTTGGTTGAGGCCTTGAGAG...
PNW87871,ATGGCAATCGTGTACGACGACACCCTGCTCGCCGCCGACTGGGTTG...
PNW88663,ATGGACCTAAATGTTGCCTTGATAGCTTGTGTGTTCCTGGTCATTG...
...,...
PNW69577,GCAGTCGCTGCTGCGGCGGCGGCGGCAGGGATAGCGGGGCTTCCCG...
PNW69575,ATGGCAGGCACATTTAAGAAGTTCTCTAAAGAGGACGTGAGCAATC...
PNW69574,ATGGCAGGCCCGGGCGGCGCGGGCGGAGGTGCTCCCTCCATGGCCG...
PNW69573,ATGGGCTGCAGCAGCAGTAAGCCCGGCGTTGTGCCGGTGCACGACT...


In [71]:
combined

,Gramene,Abundance,0,mRNA_x,mRNA_y
3055.EDO98358,PNW89085,18111.00,ATGGCCTTCGCCCTTGCCAAGTCCTCCGCTCGCGCCGCGGTGTCTC...,ATGGCCTTCGCCCTTGCCAAGTCCTCCGCTCGCGCCGCGGTGTCTC...,ATGGCCTTCGCCCTTGCCAAGTCCTCCGCTCGCGCCGCGGTGTCTC...
3055.EDP00188,PNW84742,12748.00,ATGATGCTGTCTCGCACCGTCGTCAACGTTCAGGCCAAGCTCACCA...,ATGATGCTGTCTCGCACCGTCGTCAACGTTCAGGCCAAGCTCACCA...,ATGATGCTGTCTCGCACCGTCGTCAACGTTCAGGCCAAGCTCACCA...
3055.EDP03051,PNW76070,10367.00,ATGGCCGCCATCATGAAGTCCTCCGTCCGCAGCTCCGTGCGCTCCA...,ATGGCCGCCATCATGAAGTCCTCCGTCCGCAGCTCCGTGCGCTCCA...,ATGGCCGCCATCATGAAGTCCTCCGTCCGCAGCTCCGTGCGCTCCA...
3055.EDP04241,PNW84147,8524.00,ATGGCGCGTACTGGCGCTCTACTCCTGGTCGCGCTGGCGCTTGCGG...,ATGGCGCGTACTGGCGCTCTACTCCTGGTCGCGCTGGCGCTTGCGG...,ATGGCGCGTACTGGCGCTCTACTCCTGGTCGCGCTGGCGCTTGCGG...
3055.EDP03062,PNW76019,8208.00,ATGGCGACCGCTCTGTGCAACAAGGCCTTCGCTGCCGCCCCCGTGG...,ATGGCGACCGCTCTGTGCAACAAGGCCTTCGCTGCCGCCCCCGTGG...,ATGGCGACCGCTCTGTGCAACAAGGCCTTCGCTGCCGCCCCCGTGG...
...,...,...,...,...,...
3055.EDO98310,PNW89074,0.12,ATGCCTGGCAAAAAACGAATTCGCAAGCCGAAGCATCGGCAGTCGC...,ATGCCTGGCAAAAAACGAATTCGCAAGCCGAAGCATCGGCAGTCGC...,ATGCCTGGCAAAAAACGAATTCGCAAGCCGAAGCATCGGCAGTCGC...
3055.EDO96340,PNW70381,0.08,ATGAACCTCAATGAGCGGTCAGAGGACGACTGCAAGGACTTACCGG...,ATGAACCTCAATGAGCGGTCAGAGGACGACTGCAAGGACTTACCGG...,ATGAACCTCAATGAGCGGTCAGAGGACGACTGCAAGGACTTACCGG...
3055.EDP05833,PNW77460,0.03,ATGCAACTGATGCTGGCCAAACCAAACTGGAGCGTGGCGCGGATCC...,ATGCAACTGATGCTGGCCAAACCAAACTGGAGCGTGGCGCGGATCC...,ATGCAACTGATGCTGGCCAAACCAAACTGGAGCGTGGCGCGGATCC...
3055.EDP04993,PNW83014,0.02,ATGTCGAGGCGGCGAATGGGCGGCGGGGGGGCTGGTGGGCACGCCG...,ATGTCGAGGCGGCGAATGGGCGGCGGGGGGGCTGGTGGGCACGCCG...,ATGTCGAGGCGGCGAATGGGCGGCGGGGGGGCTGGTGGGCACGCCG...


In [70]:
mRNA_list = list(combined.mRNA_y)
abundance_list = list(combined.Abundance)

abundance_list

[18111.0,
 12748.0,
 10367.0,
 8524.0,
 8208.0,
 7911.0,
 7794.0,
 7405.0,
 6691.0,
 6359.0,
 6129.0,
 5173.0,
 5008.0,
 4962.0,
 4516.0,
 4258.0,
 4004.0,
 3907.0,
 3880.0,
 3779.0,
 3503.0,
 3463.0,
 3438.0,
 3421.0,
 3194.0,
 3060.0,
 3025.0,
 2939.0,
 2923.0,
 2718.0,
 2695.0,
 2693.0,
 2679.0,
 2623.0,
 2613.0,
 2581.0,
 2569.0,
 2552.0,
 2527.0,
 2508.0,
 2499.0,
 2372.0,
 2331.0,
 2328.0,
 2261.0,
 2244.0,
 2241.0,
 2241.0,
 2216.0,
 2141.0,
 2128.0,
 2113.0,
 2093.0,
 2084.0,
 2062.0,
 2039.0,
 2039.0,
 1985.0,
 1978.0,
 1973.0,
 1966.0,
 1959.0,
 1950.0,
 1912.0,
 1904.0,
 1892.0,
 1852.0,
 1850.0,
 1848.0,
 1828.0,
 1810.0,
 1804.0,
 1772.0,
 1706.0,
 1687.0,
 1677.0,
 1662.0,
 1628.0,
 1611.0,
 1606.0,
 1568.0,
 1552.0,
 1546.0,
 1546.0,
 1537.0,
 1499.0,
 1467.0,
 1443.0,
 1429.0,
 1428.0,
 1417.0,
 1403.0,
 1378.0,
 1364.0,
 1358.0,
 1354.0,
 1339.0,
 1333.0,
 1325.0,
 1303.0,
 1293.0,
 1286.0,
 1261.0,
 1251.0,
 1249.0,
 1248.0,
 1246.0,
 1241.0,
 1239.0,
 1238.0,
 1234.0